In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [3]:
dataset = pd.read_csv("co2emission.csv")
dataset.head(100)

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.165,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.163,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Afghanistan,1945,AFG,6917471.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.129,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
96,Afghanistan,1946,AFG,7023527.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.129,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
97,Afghanistan,1947,AFG,7131209.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.129,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
98,Afghanistan,1948,AFG,7240542.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.129,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
new_data = dataset[["country", "year", "co2_per_capita"]]
new_data.head()

,country,year,co2_per_capita
0,Afghanistan,1850,NaN
1,Afghanistan,1851,NaN
2,Afghanistan,1852,NaN
3,Afghanistan,1853,NaN
4,Afghanistan,1854,NaN
